# Here we attempt to approximate pot odds using a neural network

### The first stage is to get set-up with the codebase:

## Familiarizing myself with the codebase:

In [1]:
import os
os.chdir('../../')

In [2]:
import torch
import time

In [3]:
from utils import *

In [11]:
deck = make_deck()

In [12]:
deck

['9h',
 'Qh',
 '4h',
 '7h',
 '2h',
 'Ad',
 'Qs',
 '6c',
 '9s',
 'Ks',
 'Js',
 'Ac',
 'Ah',
 '2c',
 'Ts',
 '8d',
 '5s',
 'Kh',
 'Kc',
 '3d',
 '3h',
 'Jh',
 '7s',
 '3s',
 'As',
 'Th',
 'Jd',
 'Qc',
 'Kd',
 '9d',
 '8h',
 '8s',
 '4d',
 '7c',
 'Tc',
 'Jc',
 '4s',
 '3c',
 '6h',
 '2s',
 '8c',
 '6d',
 '4c',
 '5c',
 '5d',
 '6s',
 '7d',
 '9c',
 '2d',
 'Td',
 'Qd',
 '5h']

In [13]:
def make_card_dict():
    deck = [c+s for c in CARDS for s in SUITS]
    return {deck[i]:i for i in range(len(deck))}

In [14]:
card_dict = make_card_dict()

In [15]:
def one_hot_cards(cards, card_dict=card_dict, device='cpu', dtype=torch.float):
    """
    cards is an iterable of 2 char string that shows the number and then suit
    We're going to one hot encode these cards in a 52 long vector
    """
    ret = torch.zeros(52, dtype = dtype).to(device)
    for card in cards:
        ret[card_dict[card]] = 1
    return ret

In [16]:
one_hot_cards(['8h'], card_dict).argmax() # should be 24

tensor(24)

## Naive training:
We're going to attempt to determine a likelihood of winning a heads up hand based on pocket hands through the following method.

We generate two hands and draw 5 board cards. We then determine which hand wins and assign that combination a 1 for win, and the other a 0 for loss.

We can generate a massive (1-0 balanced) training set with the above method.

Our network gets the sum of two one hot encoded vectors for our cards (basically a 2 hot encoded vector of dim 52)

We can use the treys evaluator to score our hands

In [4]:
import torch.nn as nn
import treys
from treys import Evaluator
from treys import Card

In [5]:
class simpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.mod = nn.Sequential(
            nn.Linear(52, 100),
            nn.ReLU(),
            nn.Linear(100, 500),
            nn.ReLU(),
            nn.Linear(500, 1000),
            nn.ReLU(),
            nn.Linear(1000, 1), # output layer weighted sum
            nn.Sigmoid() # if we make sigmoids we should get log_probs on the other end
        )
        self.params = self.mod.parameters()
    def forward(self, X):
        """
        X is a 52 dimensional vector (should be 2-hot encoded)
        """
        return self.mod(X)

#### Now we can begin to make the training set

In [19]:
def make_heads_up():
    """
    Makes a heads up game
    Each hand and board is a list of 2 char strings encoding the cards
    Returns: (player_1_hand, player_2_hand, board)
    """
    deck = make_deck()
    return (deck[:2], deck[2:4], deck[4:9])
    
def make_treys(cards):
    """
    Converts iterable of cards encoded as '2h' to treys.Card.new() objects
    """
    return [Card.new(c) for c in cards]

def score_heads_up(p1, p2, board):
    """
    p1, p2, board are iterables of 2char encoded cards
    returns tuple of 1 if player wins and 0 of player loses # ties are both 1
    """
    player1 = make_treys(p1)
    player2 = make_treys(p2)
    b = make_treys(board)
    e = Evaluator()
    # as ugly as the below three lines are they're the fastest method of doing this IMO.
    s1 = e.evaluate(player1, b)
    s2 = e.evaluate(player2, b)
    return (int(s1 <= s2), int(s1 >= s2))

In [20]:
p1,p2,board = make_heads_up()

In [21]:
print(p1)
print(p2)
print(board)

['Qh', '6c']
['5d', '7s']
['8c', '9s', '9c', '4d', '6s']


In [22]:
score_heads_up(p1, p2, board)

(0, 1)

In [23]:
def make_training_set(num_train = 1000, device = 'cpu', dtype=torch.float):
    X = []
    y = []
    t = {'device':device, 'dtype':dtype}
    for i in range(num_train//2): # we do //2 because we encode both sides of the game
        p1, p2, board = make_heads_up()
        s1, s2 = score_heads_up(p1, p2, board)
        X.append(one_hot_cards(p1, **t))
        X.append(one_hot_cards(p2, **t))
        y.append(torch.tensor(s1, **t))
        y.append(torch.tensor(s2, **t))
        
    return torch.stack(X), torch.stack(y).unsqueeze(1)

In [24]:
X, y = make_training_set(device='cuda')

In [25]:
X.shape

torch.Size([1000, 52])

In [26]:
y.shape

torch.Size([1000, 1])

In [27]:
model = simpleNet().to('cuda')

In [39]:
def train_model(X, y, model, epochs = 10, verbose=100):
    """
    Trains a model with MSE and Adagrad
    
    Inputs:
    X: training tensor of shape (N,52)
    y: target tensor of shape (N,1)
    model: a torch.nn.Module model
    epochs: number of epochs to train
    verbose: iterations of epochs to print out (1 for all, False for none)
    """
    optimizer = torch.optim.Adagrad(model.parameters())
    criterion = nn.MSELoss()
    
    #forward pass:
    for e in range(epochs):
        scores = model(X)
        loss = criterion(scores, y)
        with torch.no_grad():
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if verbose and e%verbose == 0:
                print("epoch: {} loss: {:.4f}".format(e,loss.item()))
                
def evaluate_model(X, y, model):
    with torch.no_grad():
        scores = model(X)
        #threshold for predictions
        scores[scores < 0.5] = 0
        scores[scores >= 0.5] = 1
        return (scores == y).sum().item()/y.shape[0]

In [40]:
train_model(X, y, model, verbose=True)

epoch: 0 loss: 0.2518
epoch: 1 loss: 0.3272
epoch: 2 loss: 0.2365
epoch: 3 loss: 0.2313
epoch: 4 loss: 0.2238
epoch: 5 loss: 0.2149
epoch: 6 loss: 0.2114
epoch: 7 loss: 0.2629
epoch: 8 loss: 0.2165
epoch: 9 loss: 0.2136


In [41]:
p1, p2, board = make_heads_up()

In [42]:
print(p1)
print(p2)
print(board)

['8c', 'Kc']
['2d', '3c']
['4s', 'Td', '3s', 'Th', '2c']


In [43]:
model(one_hot_cards(p1, device='cuda'))

tensor([0.5960], device='cuda:0', grad_fn=<SigmoidBackward>)

In [44]:
model(one_hot_cards(p2, device='cuda'))

tensor([0.2861], device='cuda:0', grad_fn=<SigmoidBackward>)

## Now that we have a naive training method lets train it on a much larger set:

In [45]:
X, y = make_training_set(num_train = 10000, device='cuda')
model = simpleNet().to('cuda')

In [49]:
train_model(X, y, model, epochs=1000, verbose = 200)

epoch: 0 loss: 0.2114
epoch: 200 loss: 0.2100
epoch: 400 loss: 0.2098
epoch: 600 loss: 0.2098
epoch: 800 loss: 0.2098


In [50]:
X_test, y_test = make_training_set(2000, device='cuda')
print("Correct: {:.2f}%".format(100*evaluate_model(X_test, y_test, model)))

Correct: 52.50%
